In [2]:
SELECT TOP(10)* FROM Sales

(10 rows affected)

Total execution time: 00:00:00.010

column1,Order_ID,Product,Quantity_Ordered,Price_Each,Order_Date,Purchase_Address,Month,Sales,City,Hour
0,295665,Macbook Pro Laptop,1,1700,2019-12-30 00:01:00.0000000,"136 Church St, New York City, NY 10001",12,1700,New York City,0
1,295666,LG Washing Machine,1,600,2019-12-29 07:03:00.0000000,"562 2nd St, New York City, NY 10001",12,600,New York City,7
2,295667,USB-C Charging Cable,1,11.949999809265137,2019-12-12 18:21:00.0000000,"277 Main St, New York City, NY 10001",12,11.949999809265137,New York City,18
3,295668,27in FHD Monitor,1,149.99000549316406,2019-12-22 15:13:00.0000000,"410 6th St, San Francisco, CA 94016",12,149.99000549316406,San Francisco,15
4,295669,USB-C Charging Cable,1,11.949999809265137,2019-12-18 12:38:00.0000000,"43 Hill St, Atlanta, GA 30301",12,11.949999809265137,Atlanta,12
5,295670,AA Batteries (4-pack),1,3.8399999141693115,2019-12-31 22:58:00.0000000,"200 Jefferson St, New York City, NY 10001",12,3.8399999141693115,New York City,22
6,295671,USB-C Charging Cable,1,11.949999809265137,2019-12-16 15:10:00.0000000,"928 12th St, Portland, OR 97035",12,11.949999809265137,Portland,15
7,295672,USB-C Charging Cable,2,11.949999809265137,2019-12-13 09:29:00.0000000,"813 Hickory St, Dallas, TX 75001",12,23.899999618530273,Dallas,9
8,295673,Bose SoundSport Headphones,1,99.98999786376953,2019-12-15 23:26:00.0000000,"718 Wilson St, Dallas, TX 75001",12,99.98999786376953,Dallas,23
9,295674,AAA Batteries (4-pack),4,2.990000009536743,2019-12-28 11:51:00.0000000,"77 7th St, Dallas, TX 75001",12,11.960000038146973,Dallas,11


## Data Exploration

In [32]:
-- Number of rows
select count(*) [Number of rows] 
from Sales

(1 row affected)

Total execution time: 00:00:00.063

Number of rows
185950


In [68]:
--Column Names
select top(0)* from Sales

(0 rows affected)

Total execution time: 00:00:00.006

Order_ID,Product,Quantity_Ordered,Price_Each,Order_Date,Purchase_Address,Month,Sales,City,Hour,Year,Day


In [49]:
--drop unwanted column: 
ALTER TABLE Sales
drop column column1

Commands completed successfully.

Total execution time: 00:00:00.012

In [51]:
-- Convert decimal of sales column
ALTER TABLE SALES
ALTER COLUMN Sales DECIMAL(10,2)

Commands completed successfully.

Total execution time: 00:00:01.664

In [53]:
-- ensure the alter
SELECT TOP(10)Sales FROM Sales

(10 rows affected)

Total execution time: 00:00:00.006

Sales
1700.00
600.00
11.95
149.99
11.95
3.84
11.95
23.90
99.99
11.96


In [56]:
--Add Year Column
ALTER TABLE SALES 
ADD Year INT
GO
UPDATE Sales
set Year = YEAR(Order_Date)

Commands completed successfully.

(178437 rows affected)

Total execution time: 00:00:01.696

In [58]:
SELECT distinct(YEAR) from sales

(2 rows affected)

Total execution time: 00:00:00.121

YEAR
2019
2020


In [60]:
-- Add Day Column
ALTER TABLE SALES 
ADD Day INT
GO
UPDATE Sales
set Day = DAY(Order_Date)

Commands completed successfully.

(178437 rows affected)

Total execution time: 00:00:02.121

In [65]:
--Add Hour column
ALTER TABLE SALES 
ADD Hour INT
GO
UPDATE Sales
set Hour = datepart(hour, Order_Date)

: Msg 2705, Level 16, State 4, Line 2
Column names in each table must be unique. Column name 'Hour' in table 'SALES' is specified more than once.

(178437 rows affected)

Total execution time: 00:00:00.444

In [28]:
select top(2)* from Sales

(2 rows affected)

Total execution time: 00:00:00.015

Order_ID,Product,Quantity_Ordered,Price_Each,Order_Date,Purchase_Address,Month,Sales,City,Hour,Year,Day,MonthName
295665,Macbook Pro Laptop,1,1700,2019-12-30 00:01:00.0000000,"136 Church St, New York City, NY 10001",12,1700.00,New York City,0,2019,30,December
295666,LG Washing Machine,1,600,2019-12-29 07:03:00.0000000,"562 2nd St, New York City, NY 10001",12,600.00,New York City,7,2019,29,December


In [25]:
--Alter table Sales
--DROP COLUMN MonthName
Alter table Sales
add MonthName VARCHAR(10)

Commands completed successfully.

Total execution time: 00:00:00.003

In [26]:
update Sales
set MonthName = FORMAT(Order_Date, 'MMMM')

(178437 rows affected)

Total execution time: 00:00:02.723

In [27]:
select distinct(MonthName)
from Sales

(12 rows affected)

Total execution time: 00:00:00.219

MonthName
September
July
August
November
April
June
March
January
May
October


## Data Cleaning

In [34]:
-- Check Null values
select
sum( case when Product is null then 1 else 0 end) as "Product",
sum( case when Quantity_Ordered is null then 1 else 0 end) as "Quantity_Ordered",
sum( case when Price_Each is null then 1 else 0 end) as "Price_Each",
sum( case when Sales is null then 1 else 0 end) as "Sales",
sum( case when Order_Date is null then 1 else 0 end) as "Order_Date",
sum( case when City is null then 1 else 0 end) as "City"
 from Sales


(1 row affected)

Total execution time: 00:00:00.168

Product,Quantity_Ordered,Price_Each,Sales,Order_Date,City
0,0,0,0,0,0


In [35]:
-- Check duplicates 
select count(*) - count(distinct(Order_ID))
from Sales

(1 row affected)

Total execution time: 00:00:00.185

(No column name)
7513


In [43]:
-- UsING a CTE with ROW_NUMBER() to identify duplicates
WITH CTE AS (
    SELECT
        S.Order_ID,
        ROW_NUMBER() OVER (PARTITION BY S.Order_ID ORDER BY Order_ID) AS RowNum
    FROM
        Sales S
)

-- Delete duplicates
DELETE FROM CTE WHERE RowNum > 1;


(7513 rows affected)

Total execution time: 00:00:00.186

In [45]:
-- Check DELECT duplicates 
select count(*) - count(distinct(Order_ID)) AS [NUMBER OF DUPLICATES]
from Sales

(1 row affected)

Total execution time: 00:00:00.125

NUMBER OF DUPLICATES
0


### Data Now is cleaned

## **Data Analysis**

In [106]:
CREATE OR ALTER PROCEDURE print2
AS
BEGIN
    SELECT TOP 2 * FROM Sales;
END;
GO
 EXEC dbo.print2

Commands completed successfully.

(2 rows affected)

Total execution time: 00:00:00.010

Order_ID,Product,Quantity_Ordered,Price_Each,Order_Date,Purchase_Address,Month,Sales,City,Hour,Year,Day,MonthName
295665,Macbook Pro Laptop,1,1700,2019-12-30 00:01:00.0000000,"136 Church St, New York City, NY 10001",12,1700.00,New York City,0,2019,30,December
295666,LG Washing Machine,1,600,2019-12-29 07:03:00.0000000,"562 2nd St, New York City, NY 10001",12,600.00,New York City,7,2019,29,December


In [108]:
SELECT sum(Sales) as [Total Sales] 
from Sales

(1 row affected)

Total execution time: 00:00:00.135

Total Sales
33437267.73


In [110]:
SELECT distinct(city)
FROM Sales

(9 rows affected)

Total execution time: 00:00:00.140

city
Boston
Seattle
Austin
Los Angeles
New York City
Atlanta
Portland
San Francisco
Dallas


In [109]:
SELECT count(Order_ID) as [Total Number of Orders] 
from Sales

(1 row affected)

Total execution time: 00:00:00.123

Total Number of Orders
178437


In [133]:
--Storde proc to out top of count to column 
CREATE OR ALTER PROC TOop_Count_with_GroupBy (
    @ColumnGroued NVARCHAR(20)
)
AS
BEGIN
    DECLARE @query NVARCHAR(MAX)
    SET @query = 
    N'SELECT 
            top(1) ' + QUOTENAME(@ColumnGroued) + N'as [Top Sales Product] 
            , COUNT(' + QUOTENAME(@ColumnGroued) + N') as [Number of Orders ],
            sum(sales) [Total Sales]
    FROM 
        SALES 
    GROUP BY '
            + QUOTENAME(@ColumnGroued) + 
    N' ORDER BY 
            2 DESC;';
    EXEC sp_executesql @query;
    RETURN;
END 
GO

Commands completed successfully.

Total execution time: 00:00:00.003

In [147]:
--Storde proc to out column and count it 
CREATE OR ALTER PROC Count_with_GroupBy (
    @ColumnGroued NVARCHAR(20)
)
AS
BEGIN
    DECLARE @query NVARCHAR(MAX)
    SET @query = 
    N'SELECT ' + QUOTENAME(@ColumnGroued) + N' ,
                COUNT(' + QUOTENAME(@ColumnGroued) + N') as [Number of Orders ] 
    FROM 
        SALES 
    GROUP BY '
            + QUOTENAME(@ColumnGroued) + 
    N' ORDER BY 
            2 DESC;';
    EXEC sp_executesql @query;
    RETURN;
END 
GO

Commands completed successfully.

Total execution time: 00:00:00.003

In [134]:
--Top product in number of sales' 
TOop_Count_with_GroupBy 'Product'

(1 row affected)

Total execution time: 00:00:00.098

Top Sales Product,Number of Orders,Total Sales
Lightning Charging Cable,20351,326223.95


In [146]:
-- products ordered by number of order 
EXEC Count_with_GroupBy 'Product';

(19 rows affected)

Total execution time: 00:00:00.110

Product,Number of Orders
Lightning Charging Cable,20351
USB-C Charging Cable,20307
AAA Batteries (4-pack),20233
AA Batteries (4-pack),20182
Wired Headphones,17677
Apple Airpods Headphones,14942
Bose SoundSport Headphones,12798
27in FHD Monitor,7348
iPhone,6532
27in 4K Gaming Monitor,6107


In [135]:
--The best city with sales
TOop_Count_with_GroupBy 'City'

(1 row affected)

Total execution time: 00:00:00.098

Top Sales Product,Number of Orders,Total Sales
San Francisco,42898,7995047.58


In [39]:
-- Cities ordered by number of order 
EXEC Count_with_GroupBy 'City'

(9 rows affected)

Total execution time: 00:00:00.065

City,Number of Orders
San Francisco,44732
Los Angeles,29605
New York City,24876
Boston,19934
Atlanta,14881
Dallas,14820
Seattle,14732
Portland,12465
Austin,9905


In [97]:
Count_with_GroupBy MonthName;

(12 rows affected)

Total execution time: 00:00:00.105

MonthName,Number of Orders
December,24004
October,19436
April,17528
November,16859
May,15836
March,14549
July,13761
June,12989
February,11496
August,11484


In [49]:
 EXEC dbo.print2

(2 rows affected)

Total execution time: 00:00:00.012

Order_ID,Product,Quantity_Ordered,Price_Each,Order_Date,Purchase_Address,Month,Sales,City,Hour,Year,Day,MonthName
295665,Macbook Pro Laptop,1,1700,2019-12-30 00:01:00.0000000,"136 Church St, New York City, NY 10001",12,1700.00,New York City,0,2019,30,December
295666,LG Washing Machine,1,600,2019-12-29 07:03:00.0000000,"562 2nd St, New York City, NY 10001",12,600.00,New York City,7,2019,29,December


In [143]:
CREATE OR ALTER PROC SalesNumbers (
    @ColumnGroued NVARCHAR(20)
)
AS
BEGIN
    DECLARE @query NVARCHAR(MAX)
    SET @query = 
        N'SELECT ' + 
        QUOTENAME(@ColumnGroued) + N' 
        , SUM( Sales ) as [Total Sales ] 
        FROM 
            SALES 
        GROUP BY '
            + QUOTENAME(@ColumnGroued) + 
        N' ORDER BY 
                2 DESC;';
    EXEC sp_executesql @query;
    RETURN;
END 
GO

Commands completed successfully.

Total execution time: 00:00:00.004

In [144]:
EXEC SalesNumbers 'City'

(9 rows affected)

Total execution time: 00:00:00.102

City,Total Sales
San Francisco,7995047.58
Los Angeles,5276437.05
New York City,4530066.13
Boston,3556583.12
Atlanta,2711195.67
Dallas,2679386.73
Seattle,2659038.32
Portland,2252113.55
Austin,1777399.58


In [105]:
EXEC SalesNumbers 'MonthName'

(12 rows affected)

Total execution time: 00:00:00.125

MonthName,Total Sales
December,4487707.44
October,3617820.95
April,3284979.66
November,3101642.69
May,3054985.37
March,2722342.53
July,2568071.67
June,2497086.94
August,2175056.91
February,2136025.10


In [55]:
EXEC SalesNumbers 'Year'

(2 rows affected)

Total execution time: 00:00:00.112

Year,Total Sales
2019,33428792.29
2020,8475.44


In [56]:
EXEC SalesNumbers 'Product'

(19 rows affected)

Total execution time: 00:00:00.140

Product,Total Sales
Macbook Pro Laptop,7872700.00
iPhone,4577300.00
ThinkPad Laptop,4033959.66
Google Phone,3157800.00
27in 4K Gaming Monitor,2387128.79
34in Ultrawide Monitor,2308439.25
Apple Airpods Headphones,2257050.00
Flatscreen TV,1417200.00
Bose SoundSport Headphones,1292370.75
27in FHD Monitor,1107976.13


In [67]:
--
CREATE OR ALTER PROC SalesQuantity (
    @ColumnGroued NVARCHAR(20)
)
AS
BEGIN
    DECLARE @query NVARCHAR(MAX)
    SET @query = 
    N'SELECT ' + QUOTENAME(@ColumnGroued) + N' , SUM( Quantity_Ordered ) as [Total Quantity Soled ]
     , SUM( Sales ) as [Total Sales ] 
    FROM SALES 
    GROUP BY '+ QUOTENAME(@ColumnGroued) + 
    N' ORDER BY 
    2 DESC, 
    3 DESC;';
    EXEC sp_executesql @query;
    RETURN;
END 
GO

Commands completed successfully.

Total execution time: 00:00:00.003

In [68]:
--products ordered by quantity
EXEC SalesQuantity 'Product'

(19 rows affected)

Total execution time: 00:00:00.135

Product,Total Quantity Soled,Total Sales
AAA Batteries (4-pack),30393,90875.07
AA Batteries (4-pack),27102,104071.68
USB-C Charging Cable,22253,265923.35
Lightning Charging Cable,21821,326223.95
Wired Headphones,19241,230699.59
Apple Airpods Headphones,15047,2257050.00
Bose SoundSport Headphones,12925,1292370.75
27in FHD Monitor,7387,1107976.13
iPhone,6539,4577300.00
27in 4K Gaming Monitor,6121,2387128.79


In [72]:
-- products of San Francisco -Which is max city with sales--
SELECT Product , 
       SUM( Quantity_Ordered ) as [Total Quantity Soled ],
       SUM( Sales ) as [Total Sales ] 
FROM 
    SALES 
WHERE
    city = 'San Francisco'
GROUP BY 
    City, 
    Product
ORDER BY 
    2 DESC, 
    3 DESC;

(19 rows affected)

Total execution time: 00:00:00.174

Product,Total Quantity Soled,Total Sales
AAA Batteries (4-pack),7270,21737.30
AA Batteries (4-pack),6418,24645.12
USB-C Charging Cable,5463,65282.85
Lightning Charging Cable,5221,78053.95
Wired Headphones,4651,55765.49
Apple Airpods Headphones,3598,539700.00
Bose SoundSport Headphones,3169,316868.31
27in FHD Monitor,1773,265932.27
iPhone,1575,1102500.00
27in 4K Gaming Monitor,1427,556515.73


### **Each city with the product have max sales prodect**

In [139]:
-- filter by total sales
Select City,
       Product AS [Product with Max Sales in the City] 
FROM(
    SELECT City,
            Product,
            RANK() OVER(PARTITION BY City 
                        ORDER BY sum(sales)) as Ranking
    FROM 
        Sales
    group by 
        City, 
        Product 
) AS Prod
where 
    Ranking = 1


(9 rows affected)

Total execution time: 00:00:00.102

City,Product with Max Sales in the City
Atlanta,AAA Batteries (4-pack)
Austin,AAA Batteries (4-pack)
Boston,AAA Batteries (4-pack)
Dallas,AAA Batteries (4-pack)
Los Angeles,AAA Batteries (4-pack)
New York City,AAA Batteries (4-pack)
Portland,AAA Batteries (4-pack)
San Francisco,AAA Batteries (4-pack)
Seattle,AAA Batteries (4-pack)


### **Each city with the most ordered product**

In [138]:
--filter by count of quantity
Select 
    City, 
    Product as [Max sales product in quantity]
FROM(
    SELECT City, Product ,
        RANK() OVER(PARTITION BY City 
                    ORDER BY count(Quantity_Ordered)) as Ranking
    FROM 
        Sales
    group by 
        City, 
        Product 
) AS Prod
where 
    Ranking = 1


(9 rows affected)

Total execution time: 00:00:00.132

City,Max sales product in quantity
Atlanta,LG Washing Machine
Austin,LG Washing Machine
Boston,LG Dryer
Dallas,LG Dryer
Los Angeles,LG Washing Machine
New York City,LG Dryer
Portland,LG Dryer
San Francisco,LG Dryer
Seattle,LG Dryer


### **Each year with the Max prodect in quantity**

In [142]:
Select 
    Year, 
    Product as [Max sales product(s) in this year]
FROM(
    SELECT Year,
           Product,
           RANK() OVER(PARTITION BY Year 
                        ORDER BY count(Quantity_Ordered)) as Ranking
    FROM 
        Sales
    group by 
         Year, 
         Product 
) AS Prod
where 
Ranking = 1

(7 rows affected)

Total execution time: 00:00:00.085

Year,Max sales product(s) in this year
2019,LG Dryer
2020,Macbook Pro Laptop
2020,27in 4K Gaming Monitor
2020,Vareebadd Phone
2020,Google Phone
2020,Wired Headphones
2020,Flatscreen TV


In [90]:
 EXEC dbo.print2

(2 rows affected)

Total execution time: 00:00:00.006

Order_ID,Product,Quantity_Ordered,Price_Each,Order_Date,Purchase_Address,Month,Sales,City,Hour,Year,Day,MonthName
295665,Macbook Pro Laptop,1,1700,2019-12-30 00:01:00.0000000,"136 Church St, New York City, NY 10001",12,1700.00,New York City,0,2019,30,December
295666,LG Washing Machine,1,600,2019-12-29 07:03:00.0000000,"562 2nd St, New York City, NY 10001",12,600.00,New York City,7,2019,29,December


### **Max Sold prodects in quantity with Days**

In [102]:
Select DISTINCT(Product) [Max Products Sale in days]
FROM(
    SELECT Day, 
            Product,
            RANK() OVER(PARTITION BY Day 
                        ORDER BY count(Quantity_Ordered)) as Ranking
    FROM 
        Sales
    group by 
        Day,
        Product 
) AS Prod
where 
    Ranking = 1


(2 rows affected)

Total execution time: 00:00:00.123

Max Products Sale in days
LG Dryer
LG Washing Machine


In [149]:
--max 4 hours with number of orders
select * 
from(
    select 
        TOP(4)hour as [Top 4 Hours in Days], 
        count(Order_ID) as [Number of Orders in]
    from 
        sales 
    GROUP BY
        Hour
    ORDER BY 
        2 DESC
) as NewTable
order by
    1

(4 rows affected)

Total execution time: 00:00:00.078

Hour,Number of Orders
11,11882
12,12082
19,12377
20,11763


In [150]:
-- max 2 days in number of orders
select 
    TOP(4)DAY as [Day], 
    count(Order_ID) as [Number of Orders]
from 
    sales 
GROUP BY
    Day
ORDER BY 
    2 DESC


(4 rows affected)

Total execution time: 00:00:00.080

Day,Number of Orders
13,6067
10,6010
2,5966
17,5931
